In [1]:
import ee
import geemap

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# Load Germany boundary
germany = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Germany'))

# Load Landsat data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2019-06-01', '2019-09-30') \
    .filterBounds(germany) \
    .filter(ee.Filter.lt('CLOUD_COVER', 30)) \
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']) \
    .median() \
    .divide(10000)

# Calculate NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Load forest carbon dataset
carbon = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1") \
    .select('agb').first().rename('Carbon') \
    .updateMask(ee.Image("UMD/hansen/global_forest_change_2023_v1_11").select('treecover2000').gt(0))

# Load other datasets
treecover = ee.Image("UMD/hansen/global_forest_change_2023_v1_11") \
    .select('treecover2000').rename('TCD')

elevation = ee.Image("USGS/SRTMGL1_003").select('elevation').rename('ELEVATION')

# Create input features
input_features = ndvi.addBands([treecover, elevation])
input_with_carbon = input_features.addBands(carbon)

# Sample points (only in forest areas)
training_points = input_with_carbon.sample(
    region=germany.geometry(),
    scale=1000,
    numPixels=3000,
    seed=42,
    dropNulls=True
)

# Split data
split = 0.7
training_data = training_points.randomColumn('random').filter(ee.Filter.lt('random', split))
testing_data = training_points.randomColumn('random').filter(ee.Filter.gte('random', split))

# Train the model
predictors = ['NDVI', 'TCD', 'ELEVATION']

trained_model = ee.Classifier.smileRandomForest(
    numberOfTrees=50,
    minLeafPopulation=5
).setOutputMode('REGRESSION').train(
    features=training_data,
    classProperty='Carbon',
    inputProperties=predictors
)

# Test the model
test = testing_data.classify(trained_model, 'predicted_carbon')

# Calculate errors
def addError(feature):
    error = ee.Number(feature.get('Carbon')).subtract(ee.Number(feature.get('predicted_carbon')))
    return feature.set({'error': error, 'error_squared': error.pow(2)})

errors = test.map(addError)

# Calculate RMSE
sum_squared_errors = errors.aggregate_sum('error_squared')
count = errors.size()
rmse = ee.Number(sum_squared_errors).divide(count).sqrt()

# Calculate R²
mean_observed = testing_data.aggregate_mean('Carbon')

def addTSS(feature):
    deviation = ee.Number(feature.get('Carbon')).subtract(mean_observed)
    return feature.set({'tss': deviation.pow(2)})

tss_features = testing_data.map(addTSS)
total_sum_squares = tss_features.aggregate_sum('tss')

residual_sum_squares = errors.aggregate_sum('error_squared')

r_squared = ee.Number(1).subtract(residual_sum_squares.divide(total_sum_squares))

# Print metrics
print('Forest Carbon Model Metrics:')
print('RMSE:', rmse.getInfo(), 'tonnes/ha')
print('R²:', r_squared.getInfo())

# Make predictions (only for forest areas)
carbon_prediction = input_features.classify(trained_model, 'predicted_carbon')

# Calculate forest carbon statistics
forest_mask = treecover.gt(25)  # Only areas with tree cover

# Corrected forest area calculation - using the correct band name
forest_area_result = forest_mask.rename('forest_area').multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=germany.geometry(),
    scale=1000,
    maxPixels=1e10
).getInfo()

forest_area_km2 = forest_area_result['forest_area'] / 1e6  # Convert to km²

stats = carbon_prediction.updateMask(forest_mask).reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.sum(),
        sharedInputs=True
    ),
    geometry=germany.geometry(),
    scale=1000,
    maxPixels=1e10
).getInfo()

mean_carbon = stats['predicted_carbon_mean']
total_carbon = stats['predicted_carbon_sum']

print("\nForest Carbon Summary:")
print("Forest Area (km²):", forest_area_km2)
print("Mean Forest Carbon (tonnes/ha):", mean_carbon)
print("Total Forest Carbon (tonnes):", total_carbon)

# Create the map
Map = geemap.Map()
Map.centerObject(germany, 6)

# Add layers
vis_params = {
    'min': 0,
    'max': 150,
    'palette': ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
}

Map.addLayer(
    carbon.clip(germany),
    vis_params,
    'Actual Forest Carbon (tonnes/ha)'
)

Map.addLayer(
    carbon_prediction.updateMask(forest_mask).clip(germany),
    vis_params,
    'Predicted Forest Carbon (tonnes/ha)'
)

# Add legend and display
Map.add_legend(
    title="Forest Carbon (tonnes/ha)",
    colors=['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837'],
    labels=['0-30', '30-60', '60-90', '90-120', '120-150']
)

Map.addLayer(
    germany.style(**{'color': 'black', 'fillColor': '00000000'}),
    {},
    'Germany Boundary'
)

display(Map)

Forest Carbon Model Metrics:
RMSE: 22.416481998794584 tonnes/ha
R²: 0.6176660129348073

Forest Carbon Summary:
Forest Area (km²): 148253.91296461044
Mean Forest Carbon (tonnes/ha): 62.98586506586478
Total Forest Carbon (tonnes): 14678298.613946483


Map(center=[51.052829262339436, 10.372114873284264], controls=(WidgetControl(options=['position', 'transparent…